In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /home/edu/.local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpotsw3ye4
  JVM stdout: /tmp/tmpotsw3ye4/h2o_edu_started_from_python.out
  JVM stderr: /tmp/tmpotsw3ye4/h2o_edu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_edu_38dx5w
H2O cluster total nodes:,1
H2O cluster free memory:,1.887 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [2]:
train_high = h2o.import_file('../cleaned-data/03-train_high.csv')
train_low = h2o.import_file('../cleaned-data/03-train_low.csv')
test_high = h2o.import_file('../cleaned-data/03-test_high.csv')
test_low = h2o.import_file('../cleaned-data/03-test_low.csv')                    

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
train_high_noid = train_high.drop(index=[0], axis=0)
train_low_noid = train_low.drop(index=[0], axis=0)
test_high_noid = test_high.drop(index=[0], axis=0)
test_low_noid = test_low.drop(index=[0], axis=0)
display(train_high.shape)
display(train_low.shape)
train_high.drop(index=[0], axis=1)

(1615, 8)

(38730, 8)

carat,cut,color,clarity,depth,table,price
2.05,4,6,1,62.4,57,18193
2.01,0,1,4,64.9,60,13744
2,1,2,3,64.2,52,11400
2,1,7,3,60.3,56,16425
2.28,4,1,4,62.7,57,16450
2.02,2,2,1,60.4,58,17357
2,3,3,3,60.7,60,15312
2.01,3,2,0,61.5,62,15475
2.31,3,2,3,62.9,58,17218
2.01,4,3,1,60.6,57,17383


In [4]:
aml = H2OAutoML(max_runtime_secs = 120, project_name = "diamonds_high", keep_cross_validation_models=True)
aml.train(x=train_high.col_names[1:-1], y = 'price', training_frame = train_high)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [5]:
pred_leader = aml.leader.predict(test_high)
pred_leader.shape

stackedensemble prediction progress: |████████████████████████████████████| 100%


(521, 1)

In [6]:
pred_df = pred_leader.as_data_frame()
pred_df = pred_df.merge(train_high['id'].as_data_frame(), left_index=True, right_index=True)
pred_df = pred_df.rename(columns={'predict':'price'})

In [7]:
pred_df[['id','price']].to_csv('../predictions/07-Auto-ML-v4-high-carat.csv', index=False)

In [8]:
pred_df

,price,id
0,16865.672589,8
1,16607.202153,34
2,15866.357655,42
3,16638.422238,72
4,15675.030633,82
...,...,...
516,14781.687216,12579
517,13467.245902,12581
518,14688.626947,12593
519,13516.743038,12609


In [9]:
aml.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200301_005708,2.82029e+06,1679.37,2.82029e+06,1304.89,0.119767
StackedEnsemble_BestOfFamily_AutoML_20200301_005708,2.8444e+06,1686.54,2.8444e+06,1308.7,0.120429
XGBoost_grid__1_AutoML_20200301_005708_model_3,2.86725e+06,1693.3,2.86725e+06,1317.9,0.120623
XGBoost_1_AutoML_20200301_005708,2.88186e+06,1697.6,2.88186e+06,1339.41,0.121938
XGBoost_3_AutoML_20200301_005708,2.89517e+06,1701.52,2.89517e+06,1350.53,0.122376
XGBoost_grid__1_AutoML_20200301_005708_model_2,2.92506e+06,1710.28,2.92506e+06,1336.35,0.121715
GBM_3_AutoML_20200301_005708,2.93492e+06,1713.16,2.93492e+06,1353.3,0.123426
XGBoost_grid__1_AutoML_20200301_005708_model_4,2.93602e+06,1713.48,2.93602e+06,1362.08,0.123042
GBM_2_AutoML_20200301_005708,2.94289e+06,1715.49,2.94289e+06,1358.31,0.123166
XGBoost_2_AutoML_20200301_005708,2.95726e+06,1719.67,2.95726e+06,1361.18,0.123872


In [10]:
aml_low = H2OAutoML(max_runtime_secs = 120, project_name = "diamonds_low", keep_cross_validation_models=True)
aml_low.train(y = 'price', x= train_low.col_names[1:-1], training_frame= train_low)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [11]:
aml_low.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200301_005915,216409,465.198,216409,259.61,0.131842
StackedEnsemble_AllModels_AutoML_20200301_005915,216409,465.198,216409,259.61,0.131842
XGBoost_1_AutoML_20200301_005915,222297,471.484,222297,266.465,0.132754
DRF_1_AutoML_20200301_005915,401245,633.439,401245,336.482,0.147085
XGBoost_2_AutoML_20200301_005915,532777,729.916,532777,437.38,0.237928
XGBoost_3_AutoML_20200301_005915,1.37537e+06,1172.76,1.37537e+06,631.627,0.262745
GLM_1_AutoML_20200301_005915,1.11052e+07,3332.45,1.11052e+07,2572.58,1.05992


In [12]:
pred_low = aml_low.leader.predict(test_low)
pred_low.shape

stackedensemble prediction progress: |████████████████████████████████████| 100%


(12928, 1)

In [13]:
pred_low.shape
type(pred_low)

h2o.frame.H2OFrame

In [14]:
pred_low = pred_low.as_data_frame()
pred_low = pd.merge(train_low['id'].as_data_frame(), pred_low, left_index=True, right_index=True)
pred_low = pred_low.rename(columns={'predict':'price'})
pred_low[['id','price']].to_csv('../predictions/07-Auto-ML-v4-low-carat.csv', index=False)

In [15]:
display(pred_df, pred_low)
pred_low[:10]

,price,id
0,16865.672589,8
1,16607.202153,34
2,15866.357655,42
3,16638.422238,72
4,15675.030633,82
...,...,...
516,14781.687216,12579
517,13467.245902,12581
518,14688.626947,12593
519,13516.743038,12609


,id,price
0,0,4381.571444
1,1,1149.469952
2,2,6929.500714
3,3,1552.995621
4,4,2183.149034
...,...,...
12923,13475,709.136603
12924,13477,2483.153581
12925,13478,558.059120
12926,13479,826.804274


,id,price
0,0,4381.571444
1,1,1149.469952
2,2,6929.500714
3,3,1552.995621
4,4,2183.149034
5,5,1982.491933
6,6,1873.289710
7,7,3448.970372
8,9,14542.539019
9,10,3566.534343


In [16]:
test_pred = pd.concat([pred_df, pred_low],ignore_index=True)
test_pred.head()

,price,id
0,16865.672589,8
1,16607.202153,34
2,15866.357655,42
3,16638.422238,72
4,15675.030633,82


In [17]:
test_pred2 = pd.concat([test_pred.id, test_pred.price], axis=1)

In [18]:
#test_pred.to_csv('../predictions/8-Auto-ML-Split-data-v1.csv', index=False)

In [19]:
test_pred2 = test_pred2.sort_values(by='id')
test_pred['id'] = test_pred2.id.map(str)

In [20]:
test_pred2.to_csv('../predictions/8-Auto-ML-Split-data-v1.csv', index=False)

In [21]:
test_pred2[:10]

,id,price
521,0,4381.571444
522,1,1149.469952
523,2,6929.500714
524,3,1552.995621
525,4,2183.149034
526,5,1982.491933
527,6,1873.289710
528,7,3448.970372
0,8,16865.672589
529,9,14542.539019
